# Acrobot

### Описание 
Система состоит из двух звеньев, соединенных линейно в цепь, один конец которой закреплен. Соединение между двумя звеньями приводится в действие. Цель состоит в том, чтобы приложить крутящий момент к приводимому в действие соединению, чтобы повернуть свободный конец линейной цепи выше заданной высоты, начиная с начального состояния свисания вниз.


https://gymnasium.farama.org/environments/classic_control/acrobot/



Как видно на **Gif**: два синих тростя, соединенные двумя зелеными "шарнирами". Происходит движение между двумя звеньями. Цель состоит в том, чтобы раскачать свободный конец внешнего звена. для достижения заданной высоты (черная горизонтальная линия над системой) путем приложения крутящего момента к приводу.

### Возможные действия

    Действие является дискретным, детерминированным и представляет собой крутящий момент, приложенный к приводимому 
    в действие шарниру между двумя звеньями.
    
    | Ном | Действие                                             | Еденица измерения     |
    |-----|------------------------------------------------------|-----------------------|
    | 0   | применить крутящий момент -1 к приводному соединению | крутящий момент (Н·м) |
    | 1   | применить крутящий момент 0 к приводному соединению  | крутящий момент (Н·м) |
    | 2   | применить крутящий момент 1 к приводному соединению  | крутящий момент (Н·м) |

 ### Параметры
 
 Парметры представляют собой массив «ndarray» формы «(6,)», который включает информацию о двух углах поворота шарнира,
 а также их угловые скорости:

    | Ном | Параметры                    | Мин                 | Макс              |
    |-----|------------------------------|---------------------|-------------------|
    | 0   | Косинус `theta1`             | -1                  | 1                 |
    | 1   | Синус  `theta1`              | -1                  | 1                 |
    | 2   | Косинус `theta2`             | -1                  | 1                 |
    | 3   | Синус  `theta2`              | -1                  | 1                 |
    | 4   | Угловая скоорость `theta1`   | ~ -12.567 (-4 * pi) | ~ 12.567 (4 * pi) |
    | 5   | Угловая скоорость `theta2`   | ~ -28.274 (-9 * pi) | ~ 28.274 (9 * pi) |


- `theta1` - это угол первого соединения, где значение 0 указывает, что первый трость направлен прямо вниз.

- `theta2` ***относительно угла первого тростя.*** Угол 0 соответствует одинаковому углу между двумя звеньями.

Угловые скорости `theta1` и `theta2` ограничены ±4π и ±9π рад/с соответственно. Состояние `[1, 0, 1, 0, ..., ...]` указывает, что обе ссылки направлены вниз.

### Вознограждение

Цель состоит в том, чтобы свободный конец достиг заданной целевой высоты за как можно меньшее количество шагов. и поэтому все шаги, которые не достигают цели, получают вознаграждение в размере -1. Достижение целевой высоты приводит к завершению с вознаграждением ***0***. Порог вознаграждения равен -100.

### Начальное состояние

Каждый параметр в базовом состоянии ("theta1", "theta2" и две угловые скорости) инициализируется равномерно между -0,1 и 0,1. Это означает, что обе связи направлены вниз с некоторой начальной стохастичностью.

### Завершение эпизода

Эпизод заканчивается, если происходит одно из следующих событий: 
1. Завершение: свободный конец достигает заданной высоты, которая строится как: `-cos(theta1) - cos(theta2 + theta1) > 1.0` 
2. Прекращение действий: количество ходов больше `500 (200 для v0)` 

In [ ]:
from random import randint

### Источник агента
https://github.com/openai/gym/blob/master/gym/envs/classic_control/acrobot.py

In [ ]:
import gym
"""classic Acrobot task"""
from typing import Optional

import numpy as np
from numpy import cos, pi, sin

from gym import core, logger, spaces
from gym.error import DependencyNotInstalled

In [2]:
env = gym.make("Acrobot-v1")
copyright__ = "Copyright 2013, RLPy http://acl.mit.edu/RLPy"
__credits__ = [
    "Alborz Geramifard",
    "Robert H. Klein",
    "Christoph Dann",
    "William Dabney",
    "Jonathan P. How",
]
__license__ = "BSD 3-Clause"
__author__ = "Christoph Dann <cdann@cdann.de>"

### Используем класс AcrobotEnv

In [3]:
from gym.envs.classic_control import utils

class AcrobotEnv(core.Env):

    metadata = {
        "render_modes": ["human", "rgb_array"],
        "render_fps": 15,
    }

    dt = 0.2

    LINK_LENGTH_1 = 1.0  # [m]
    LINK_LENGTH_2 = 1.0  # [m]
    LINK_MASS_1 = 1.0  #: [kg] mass of link 1
    LINK_MASS_2 = 1.0  #: [kg] mass of link 2
    LINK_COM_POS_1 = 0.5  #: [m] position of the center of mass of link 1
    LINK_COM_POS_2 = 0.5  #: [m] position of the center of mass of link 2
    LINK_MOI = 1.0  #: moments of inertia for both links

    MAX_VEL_1 = 4 * pi
    MAX_VEL_2 = 9 * pi

    AVAIL_TORQUE = [-1.0, 0.0, +1]

    torque_noise_max = 0.0

    SCREEN_DIM = 500

    #: use dynamics equations from the nips paper or the book
    book_or_nips = "book"
    action_arrow = None
    domain_fig = None
    actions_num = 3

    def __init__(self, render_mode: Optional[str] = None):
        self.render_mode = render_mode
        self.screen = None
        self.clock = None
        self.isopen = True
        high = np.array(
            [1.0, 1.0, 1.0, 1.0, self.MAX_VEL_1, self.MAX_VEL_2], dtype=np.float32
        )
        low = -high
        self.observation_space = spaces.Box(low=low, high=high, dtype=np.float32)
        self.action_space = spaces.Discrete(3)
        self.state = None

    def reset(self, *, seed: Optional[int] = None, options: Optional[dict] = None):
        super().reset(seed=seed)
        # Note that if you use custom reset bounds, it may lead to out-of-bound
        # state/observations.
        low, high = utils.maybe_parse_reset_bounds(
            options, -0.1, 0.1  # default low
        )  # default high
        self.state = self.np_random.uniform(low=low, high=high, size=(4,)).astype(
            np.float32
        )

        if self.render_mode == "human":
            self.render()
        return self._get_ob(), {}

    def step(self, a):
        s = self.state
        #print(f' это s: {s}')
        assert s is not None, "Call reset before using AcrobotEnv object."
        torque = self.AVAIL_TORQUE[a]

        # Add noise to the force action
        if self.torque_noise_max > 0:
            torque += self.np_random.uniform(
                -self.torque_noise_max, self.torque_noise_max
            )

        # Now, augment the state with our force action so it can be passed to
        # _dsdt
        s_augmented = np.append(s, torque)

        ns = rk4(self._dsdt, s_augmented, [0, self.dt])

        ns[0] = wrap(ns[0], -pi, pi)
        ns[1] = wrap(ns[1], -pi, pi)
        ns[2] = bound(ns[2], -self.MAX_VEL_1, self.MAX_VEL_1)
        ns[3] = bound(ns[3], -self.MAX_VEL_2, self.MAX_VEL_2)
        self.state = ns
        terminated = self._terminal()
        reward = -1.0 if not terminated else 0.0

        if self.render_mode == "human":
            self.render()
        return (self._get_ob(), reward, terminated, False, {})

    def _get_ob(self):
        s = self.state
        assert s is not None, "Call reset before using AcrobotEnv object."
        return np.array(
            [cos(s[0]), sin(s[0]), cos(s[1]), sin(s[1]), s[2], s[3]], dtype=np.float32
        )

    def _terminal(self):
        s = self.state
        assert s is not None, "Call reset before using AcrobotEnv object."
        return bool(-cos(s[0]) - cos(s[1] + s[0]) > 1.0)

    def _dsdt(self, s_augmented):
        m1 = self.LINK_MASS_1
        m2 = self.LINK_MASS_2
        l1 = self.LINK_LENGTH_1
        lc1 = self.LINK_COM_POS_1
        lc2 = self.LINK_COM_POS_2
        I1 = self.LINK_MOI
        I2 = self.LINK_MOI
        g = 9.8
        a = s_augmented[-1]
        s = s_augmented[:-1]
        theta1 = s[0]
        theta2 = s[1]
        dtheta1 = s[2]
        dtheta2 = s[3]
        d1 = (
            m1 * lc1**2
            + m2 * (l1**2 + lc2**2 + 2 * l1 * lc2 * cos(theta2))
            + I1
            + I2
        )
        d2 = m2 * (lc2**2 + l1 * lc2 * cos(theta2)) + I2
        phi2 = m2 * lc2 * g * cos(theta1 + theta2 - pi / 2.0)
        phi1 = (
            -m2 * l1 * lc2 * dtheta2**2 * sin(theta2)
            - 2 * m2 * l1 * lc2 * dtheta2 * dtheta1 * sin(theta2)
            + (m1 * lc1 + m2 * l1) * g * cos(theta1 - pi / 2)
            + phi2
        )
        if self.book_or_nips == "nips":
            # the following line is consistent with the description in the
            # paper
            ddtheta2 = (a + d2 / d1 * phi1 - phi2) / (m2 * lc2**2 + I2 - d2**2 / d1)
        else:
            # the following line is consistent with the java implementation and the
            # book
            ddtheta2 = (
                a + d2 / d1 * phi1 - m2 * l1 * lc2 * dtheta1**2 * sin(theta2) - phi2
            ) / (m2 * lc2**2 + I2 - d2**2 / d1)
        ddtheta1 = -(d2 * ddtheta2 + phi1) / d1
        return dtheta1, dtheta2, ddtheta1, ddtheta2, 0.0

    def render(self):
        if self.render_mode is None:
            logger.warn(
                "You are calling render method without specifying any render mode. "
                "You can specify the render_mode at initialization, "
                f'e.g. gym("{self.spec.id}", render_mode="rgb_array")'
            )
            return

        try:
            import pygame
            from pygame import gfxdraw
        except ImportError:
            raise DependencyNotInstalled(
                "pygame is not installed, run `pip install gym[classic_control]`"
            )

        if self.screen is None:
            pygame.init()
            if self.render_mode == "human":
                pygame.display.init()
                self.screen = pygame.display.set_mode(
                    (self.SCREEN_DIM, self.SCREEN_DIM)
                )
            else:  # mode in "rgb_array"
                self.screen = pygame.Surface((self.SCREEN_DIM, self.SCREEN_DIM))
        if self.clock is None:
            self.clock = pygame.time.Clock()

        surf = pygame.Surface((self.SCREEN_DIM, self.SCREEN_DIM))
        surf.fill((255, 255, 255))
        s = self.state

        bound = self.LINK_LENGTH_1 + self.LINK_LENGTH_2 + 0.2  # 2.2 for default
        scale = self.SCREEN_DIM / (bound * 2)
        offset = self.SCREEN_DIM / 2

        if s is None:
            return None

        p1 = [
            -self.LINK_LENGTH_1 * cos(s[0]) * scale,
            self.LINK_LENGTH_1 * sin(s[0]) * scale,
        ]

        p2 = [
            p1[0] - self.LINK_LENGTH_2 * cos(s[0] + s[1]) * scale,
            p1[1] + self.LINK_LENGTH_2 * sin(s[0] + s[1]) * scale,
        ]

        xys = np.array([[0, 0], p1, p2])[:, ::-1]
        thetas = [s[0] - pi / 2, s[0] + s[1] - pi / 2]
        link_lengths = [self.LINK_LENGTH_1 * scale, self.LINK_LENGTH_2 * scale]

        pygame.draw.line(
            surf,
            start_pos=(-2.2 * scale + offset, 1 * scale + offset),
            end_pos=(2.2 * scale + offset, 1 * scale + offset),
            color=(0, 0, 0),
        )

        for ((x, y), th, llen) in zip(xys, thetas, link_lengths):
            x = x + offset
            y = y + offset
            l, r, t, b = 0, llen, 0.1 * scale, -0.1 * scale
            coords = [(l, b), (l, t), (r, t), (r, b)]
            transformed_coords = []
            for coord in coords:
                coord = pygame.math.Vector2(coord).rotate_rad(th)
                coord = (coord[0] + x, coord[1] + y)
                transformed_coords.append(coord)
            gfxdraw.aapolygon(surf, transformed_coords, (0, 204, 204))
            gfxdraw.filled_polygon(surf, transformed_coords, (0, 204, 204))

            gfxdraw.aacircle(surf, int(x), int(y), int(0.1 * scale), (204, 204, 0))
            gfxdraw.filled_circle(surf, int(x), int(y), int(0.1 * scale), (204, 204, 0))

        surf = pygame.transform.flip(surf, False, True)
        self.screen.blit(surf, (0, 0))

        if self.render_mode == "human":
            pygame.event.pump()
            self.clock.tick(self.metadata["render_fps"])
            pygame.display.flip()

        elif self.render_mode == "rgb_array":
            return np.transpose(
                np.array(pygame.surfarray.pixels3d(self.screen)), axes=(1, 0, 2)
            )

    def close(self):
        if self.screen is not None:
            import pygame

            pygame.display.quit()
            pygame.quit()
            self.isopen = False


#### def wrap(x, m, M):
"""Переносит ``x`` таким образом, что m <= x <= M; но в отличие от ``bound()``, который сокращает, ``wrap()`` оборачивает x вокруг системы координат, заданной m,M.\n Например, m = -180, M = 180 (градусов), x = 360 --> возвращает 0.

* Аргументы:
          х: скаляр
          m: минимально возможное значение в диапазоне
          M: максимально возможное значение в диапазоне
* Возвращает:
           x: скаляр, "обернутый"

In [ ]:
def wrap(x, m, M):
    diff = M - m
    while x > M:
        x = x - diff
    while x < m:
        x = x + diff
    return x

#### def bound(x, m, M=None):

* Либо берёт m как скаляр, тогда  bound(x, m, M), возвращает m <= x <= M *OR*
*      либо m вектор длины 2, bound(x,m, <IGNORED>) возвращает m[0] <= x <= m[1].

* Аргументы:
          x: скалярное значение
          m: нижняя граница
          M: верхняя граница
* Возаращет: скалярное значение, ограниченное между min (m) и Max (M)

In [ ]:
def bound(x, m, M=None):
    if M is None:
        M = m[1]
        m = m[0]
    return min(max(x, m), M)

In [4]:
def rk4(derivs, y0, t):
    """
    Integrate 1-D or N-D system of ODEs using 4-th order Runge-Kutta.

    Example for 2D system:

        >>> def derivs(x):
        ...     d1 =  x[0] + 2*x[1]
        ...     d2 =  -3*x[0] + 4*x[1]
        ...     return d1, d2

        >>> dt = 0.0005
        >>> t = np.arange(0.0, 2.0, dt)
        >>> y0 = (1,2)
        >>> yout = rk4(derivs, y0, t)

    Args:
        derivs: the derivative of the system and has the signature ``dy = derivs(yi)``
        y0: initial state vector
        t: sample times

    Returns:
        yout: Runge-Kutta approximation of the ODE
    """

    try:
        Ny = len(y0)
    except TypeError:
        yout = np.zeros((len(t),), np.float_)
    else:
        yout = np.zeros((len(t), Ny), np.float_)

    yout[0] = y0

    for i in np.arange(len(t) - 1):

        this = t[i]
        dt = t[i + 1] - this
        dt2 = dt / 2.0
        y0 = yout[i]

        k1 = np.asarray(derivs(y0))
        k2 = np.asarray(derivs(y0 + dt2 * k1))
        k3 = np.asarray(derivs(y0 + dt2 * k2))
        k4 = np.asarray(derivs(y0 + dt * k3))
        yout[i + 1] = y0 + dt / 6.0 * (k1 + 2 * k2 + 2 * k3 + k4)
    # We only care about the final timestep and we cleave off action value which will be zero
    return yout[-1][:4]


### Функция обучения нашего агента

1. Функция на вход получает список "ходов", состоящий соответственно из 0, 1 и 2
2. Циклом она обходит это список, получает из него цифры, и совершает по ним ходы (acrobot.step()) для модели Acrobot 
3. Получает результат, -1 означает что высота не достигнута, и 0 означает, что трость достигла нужной высоты
4. В том случае если мы получаем 0, мы берём всю комбинацию цифр, которые позволили нам достичь успеха из на шего списка list_step, в  новый список kit
5. Затем выполняем acrobot.reset(), останавливаем нашу "игру", и выполняем выход из списка
6. По итогу функция возвращает либо набор цифр "победной" комбинации, либо тот список который подавался ей на вход изнчально  

In [ ]:
def education(list_step):
    kit = list_step
    number = 1
    for move in list_step:
        step = acrobot.step(move)
        result = step[1]
        if result != -1.0:
            if number < len(kit):
                kit = list_step[0:number]
                acrobot.reset()
                break   
        number += 1  
    return kit

##### Создаём экземпляр класса AcrobotEnv(), и как указано по инструкции, перед первым ходом выполняеем reset

In [5]:
acrobot = AcrobotEnv()
acrobot.reset()

#### Обучение агента

1. Для начала мы сосздаём список "list_step" из произволных чисел, диапазона от 0 до 2 включительно (варинаты возможных ходов из инструкции), количество цифр будет равно 200 (максимальное количество ходов, для версии v0)
2. Затем создаём список "winning_combination" в котором будем хранить самую короткую "победную" комбинацию которую научиться делать наш агент
3. Мы 100 раз обучаем нашего агента, подаем на вход "функции Обучения" список "list_step", и функция либо возвращает нам его (в том случае когда цифрами из списка выиграть не возможно за 200 шагов), либо при получении вознаграждения 0, возращает список который меньше "list_step".
4. Затем мы последовательно сравниваем  все списки "победных комбинаций", и выбираем из них самую короткую
5. В конце мы выводим комбинацию, которая позволит нам получить вознагрождение 0, за наименьшее количества ходов

In [273]:
list_step = [randint(0, 2) for i in range(200)]
winning_combination = list_step
for iteration in range(0, 100):
    combination = education(list_step)
    if type(combination) == list:
        if len(combination) < len(winning_combination):
            winning_combination = combination
                  
print(f'агент научился вполнять задачу за: {len(winning_combination)} шагов')
print(winning_combination)

агент научился вполнять задачу за: 26 шагов
[0, 0, 1, 0, 1, 2, 0, 1, 2, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 2, 1, 2, 2, 1, 2, 0]
